# 浙教必修二 4.2搭建信息系统


> 行空板本身相当于一台能接入开源硬件的linux电脑，因此可以将服务器端及传感器数据发送都在行空板上使用纯Python代码来实现。

# 使用的技术点

- flask库实现web服务器的搭建
- sqlite库实现数据库的操作
- request库实现数据的传输
- pinpong库实现传感器数据的读取
- unihiker库实现行空板屏幕的显示

# 操作步骤

## 1-数据库初始化

首先运行一次如下代码初始化sqlite库，生成数据库文件

In [ ]:
import sqlite3
import datetime
DATABASE = 'data/data.db'
def setup_db():
    db = sqlite3.connect(DATABASE)
    cur = db.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS sensorlist(sensorid INTEGER PRIMARY KEY autoincrement, sensorname text,maxvalue int,minvalue int)")
    cur.execute("CREATE TABLE IF NOT EXISTS sensorlog(logid INTEGER PRIMARY KEY autoincrement , sensorid int,sensorvalue float,updatetime time)")
    db.commit()
    cur.execute("SELECT COUNT(*) FROM sensorlist")
    if cur.fetchall()[0][0] == 0:
        cur.execute('INSERT INTO sensorlist(sensorid,sensorname,maxvalue,minvalue) VALUES(1,"温度传感器",39,5)')
        cur.execute('INSERT INTO sensorlist(sensorid,sensorname,maxvalue,minvalue) VALUES(2,"湿度传感器",80,20)')
        db.commit()
    cur.execute("SELECT COUNT(*) FROM sensorlog")
    now = datetime.datetime.now()
    now = now.strftime('%Y-%m-%d %H:%M:%S')
    if cur.fetchall()[0][0] == 0:
        cur.execute("INSERT INTO sensorlog(logid,sensorid,sensorvalue,updatetime) VALUES(1,1,39,'%s')"%now)
        db.commit()
if __name__ == "__main__":
  setup_db()
  print("初始化完毕")


## 2-开启网页服务器

运行如下代码，在行空板开启一个网页服务，开启之后可以浏览器访问行空板IP:端口即可网页查看数据。
例如使用USB线连接时可以访问: http://10.1.2.3:8090 
> 注：行空板上8080端口默认给siot使用，因此此处使用8090端口

In [ ]:
# coding= UTF-8
import sqlite3
import datetime
import json
from flask import Flask,render_template, request
DATABASE = 'data/data.db'
app = Flask(__name__)

IP="10.1.2.3"
PORT=8000

@app.route("/")
def hello():
    db = sqlite3.connect(DATABASE)
    cur = db.cursor()
    cur.execute("SELECT * FROM sensorlog WHERE sensorid =1")
    data = cur.fetchall()
    cur.close()
    db.close()
    temp1 = data[len(data) - 1]
    temp=temp1[2]
    return render_template('vews.html', data=data,temp=temp)

#Get data
@app.route("/get",methods=['GET'])
def get_data():
    sensorid=int(request.args.get('id'))
    db = sqlite3.connect(DATABASE)
    cur = db.cursor()
    cur.execute("SELECT * FROM sensorlog WHERE sensorid = %s"% sensorid)
    data = cur.fetchall()
    dbsum=len(data)
    dset={'sensorid':sensorid}
    temp=[]
    for i in range(dbsum):
        value={}
        value['sensorvalue']=data[i][2]
        value['updatetime']=data[i][3]
        temp.append(value)
        #dset['value']={'sensorvalue':data[i-1][2],'updatetime':data[i-1][3]}
    dset['value']=temp
    djson=json.dumps(dset)
    return djson

#Adding data
@app.route("/input",methods=['POST','GET'])
def add_data():
    if request.method == 'POST':
        sensorid = int(request.form.get('id'))
        sensorvalue = float(request.form.get('val'))
    else:
        sensorid = int(request.args.get('id'))
        sensorvalue = float(request.args.get('val')) 
    nowtime = datetime.datetime.now()
    nowtime = nowtime.strftime('%Y-%m-%d %H:%M:%S')
    db = sqlite3.connect(DATABASE)
    cur = db.cursor()
    cur.execute("INSERT INTO sensorlog(sensorid,sensorvalue,updatetime) VALUES(%d,%f,'%s')" %(sensorid,sensorvalue,nowtime) )
    db.commit()
    cur.execute("SELECT * FROM sensorlist where sensorid = %d"% sensorid)
    rv = cur.fetchall()
    cur.close()
    db.close()
    maxrv = rv[0][2]
    minrv = rv[0][3]
    if sensorvalue > maxrv or sensorvalue < minrv:
        return '1'
    else:
        return '0'

if __name__ == "__main__":
    app.run(host=IP, port=PORT,debug=False,threaded=True) #jupyter中运行要关闭debug


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://10.1.2.3:8000/ (Press CTRL+C to quit)
10.1.2.3 - - [16/Jul/2022 02:08:41] "GET /input?id=1&val=0 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:42] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:43] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:44] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:46] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:47] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:48] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:49] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.101 - - [16/Jul/2022 02:08:49] "GET / HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:50] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:51] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.101 - - [16/Jul/2022 02:08:52] "GET / HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:08:52] "GET /input?id=1&val=24 

10.1.2.3 - - [16/Jul/2022 02:10:33] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:34] "GET /input?id=1&val=16 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:35] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:36] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:37] "GET /input?id=1&val=20 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:38] "GET /input?id=1&val=19 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:39] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:40] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:41] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:42] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:43] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:45] "GET /input?id=1&val=24 HTTP/1.1" 200 -
10.1.2.3 - - [16/Jul/2022 02:10:46] "GET /input?id=1&val=16 HTTP/1.1" 200 -
10.1.2.3 - -